In [1]:
import json
from tqdm import tqdm
import random

In [2]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/sft_data/llava62k_v4.json", 'r', encoding='utf-8') as file:
    dpo_data = json.load(file)
print(len(dpo_data))

62461


In [3]:
dpo_data[1000]

{'id': '000000064933',
 'image': 'coco/train2017/000000064933.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the woman doing in the image?'},
  {'from': 'gpt',
   'value': 'The woman in the image is playing tennis, standing on a tennis court, and about to hit a tennis ball with her racket.'},
  {'from': 'human',
   'value': 'How many images are there of the woman with the tennis racket?'},
  {'from': 'gpt',
   'value': 'There is a single image that features four photos of the woman in different poses with a tennis racket.'},
  {'from': 'human',
   'value': 'Is the woman standing in the field or on the tennis court?'},
  {'from': 'gpt',
   'value': 'The woman is standing on a tennis court while playing with a tennis ball.'},
  {'from': 'human', 'value': 'What is the woman using to play tennis?'},
  {'from': 'gpt',
   'value': 'The woman is using a tennis racket to play tennis.'},
  {'from': 'human',
   'value': 'Can you provide a brief overview of the sport of 

In [4]:
"""
textvqa 数据集全部都是以 <image>\nProvide a one-sentence caption for the provided image.\n 作为问题的开头
"""
count = 0
count_convs = 0
textvqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'textvqa':
        textvqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
        if len(data["conversations"])==2:
            count_convs += 1
print(count)
print(count_convs)
print(len(textvqa_data))

2196
2196
2196


In [5]:
"""
coco 数据集全部都是以 <image>\n 或者是 <image> 作为问题的开头或者是结尾
"""
count = 0
coco_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'coco':
        coco_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
        if data["conversations"][0]['value'].endswith('<image>'):
            count+=1
print(count)
print(len(coco_data))

36410
36410


In [6]:
"""
ocr_vqa 全部都是以 <image>\n 作为问题的开头
"""
count = 0
ocr_vqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'ocr_vqa':
        ocr_vqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n'):
            count+=1
print(count)
print(len(ocr_vqa_data))

8000
8000


In [7]:
"""
gpa 全部都是以 <image>\n 作为问题的开头
"""
count = 0
statistic = 0
gqa_data = []
for data in dpo_data:
    if data["image"].split('/')[0] == 'gqa':
        gqa_data.append(data)
        if data["conversations"][0]['value'].startswith('<image>\n') and data["conversations"][0]['value'].endswith('Answer the question using a single word or phrase.'):
            count+=1
print(count)
print(len(gqa_data))

7214
7214


In [8]:
def find_companions(input_str, str_list, count=1):
    filtered_list = [s for s in str_list if s != input_str]
    return random.sample(filtered_list, count)

In [9]:
textvqa_image = []
for data in textvqa_data:
    textvqa_image.append(data["image"])
print(len(textvqa_image))

coco_image = []
for data in coco_data:
    coco_image.append(data["image"])
print(len(coco_image))

ocr_vqa_image = []
for data in ocr_vqa_data:
    ocr_vqa_image.append(data["image"])
print(len(ocr_vqa_image))

# vq_image = []
# for data in vg_data:
#     vq_image.append(data["image"])
# print(len(vq_image))

gqa_image = []
for data in gqa_data:
    gqa_image.append(data["image"])
print(len(gqa_image))

all_images = textvqa_image + coco_image + ocr_vqa_image + gqa_image
print(len(all_images))

2196
36410
8000
7214
53820


In [11]:
from PIL import Image

def resize_and_paste_image(image1_path, image2_path, output_path):
    # 打开两张图片
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)

    # 获取第二张图片的尺寸
    image2_width, image2_height = image2.size

    # 计算将第一张图片缩小为第二张图片大小的一半，同时保持比例
    aspect_ratio = image1.width / image1.height
    if image2_width * 0.7 / aspect_ratio <= image2_height * 0.7:
        new_width = int(image2_width * 0.7)
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = int(image2_height * 0.7)
        new_width = int(new_height * aspect_ratio)

    image1_resized = image1.resize((new_width, new_height))

    # 计算将第一张图片放在第二张图片中间的坐标
    paste_x = (image2_width - new_width) // 2
    paste_y = (image2_height - new_height) // 2

    # 在第二张图片的中间粘贴第一张图片
    image2.paste(image1_resized, (paste_x, paste_y))

    image2 = image2.convert('RGB')

    # 保存结果
    image2.save(output_path)

# 调用函数
# resize_and_paste_image('/mnt/petrelfs/liuziyu/RLHF/Observation/pics/bear.png', '/mnt/petrelfs/liuziyu/RLHF/Observation/pics/medicine.png', 'output_image1.png')


In [12]:
meta_prompt = [
    'For the image in the center',
    'Regarding the central image',
    'Concerning the middle image',
    'Pertaining to the central picture',
    'With respect to the image in the middle',
    'As for the image at the center',
    'Focusing on the central image',
    'In relation to the middle picture',
    'When it comes to the central image',
    'With regards to the image in the center',
    'For the image located in the center',
    'About the central image',
    'Referring to the image in the middle',
    'In terms of the central picture',
    'For the middle picture',
    'Regarding the image in the middle',
    'Concerning the image at the center',
    'Pertaining to the image in the center',
    'With respect to the central picture',
    'As for the image in the middle',
    'Focusing on the middle image',
    'In relation to the central image',
    'When it comes to the image in the center',
    'With regards to the central picture',
    'For the picture in the center',
    'Regarding the middle image',
    'Concerning the picture in the middle',
    'Pertaining to the middle image',
    'With respect to the image at the center',
    'As for the central picture',
    'Focusing on the image in the center',
    'In relation to the image in the middle',
    'When it comes to the central picture',
    'With regards to the middle image',
    'For the central picture',
    'Regarding the image in the center',
    'Concerning the image in the middle',
    'Pertaining to the central image',
    'With respect to the middle picture',
    'As for the picture in the center',
    'Focusing on the central picture',
    'In relation to the middle image',
    'When it comes to the image at the center',
    'With regards to the image in the middle',
    'For the image at the center',
    'Regarding the central picture',
    'Concerning the central image',
    'Pertaining to the picture in the center',
    'With respect to the central image',
    'As for the image in the center', 
    'Regarding the central subplot',
    'Concerning the central subplot',
    'Pertaining to the central subplot',
    'With respect to the central subplot',
    'As for the central subplot',
    'Focusing on the central subplot',
    'In relation to the central subplot',
    'When it comes to the central subplot',
    'With regards to the central subplot',
    'For the subplot in the center',
    'About the central subplot',
    'Referring to the central subplot',
    'In terms of the central subplot',
    'For the subplot at the center',
    'Regarding the subplot in the center',
    'Concerning the subplot in the middle',
    'Pertaining to the subplot in the center',
    'With respect to the subplot at the center',
    'As for the subplot in the middle',
    'Focusing on the subplot at the center',
    'In relation to the subplot in the middle',
    'When it comes to the subplot in the center',
    'With regards to the subplot at the center',
    'For the subplot located in the center',
    'Regarding the middle subplot',
    'Concerning the subplot in the center',
    'Pertaining to the middle subplot',
    'With respect to the subplot in the middle',
    'As for the middle subplot',
    'Focusing on the middle subplot',
    'In relation to the middle subplot',
    'When it comes to the middle subplot',
    'With regards to the middle subplot',
    'For the central plot',
    'Regarding the plot in the middle',
    'Concerning the plot at the center',
    'Pertaining to the central plot',
    'With respect to the central plot',
    'As for the plot in the center',
    'Focusing on the plot in the middle',
    'In relation to the plot at the center',
    'When it comes to the plot in the middle',
    'With regards to the central plot',
]

In [13]:
mix_textvqa_data = []
for index, data in tqdm(enumerate(textvqa_data)):
    
    image_list = find_companions(data['image'], textvqa_image, 1)
    in_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + data['image']
    out_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + image_list[0]

    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_pics_in_pics/textvqa_{index}.jpg'
    resize_and_paste_image(in_pics, out_pics, output_path)

    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = '<image>\n' + local_meta_prompt + ', ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = local_meta_prompt + ', ' +  convs['value'][0].lower() + convs['value'][1:]

    data['image'] = output_path
    mix_textvqa_data.append(data)

print(len(mix_textvqa_data))
output_json_path = "./mix_textvqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_textvqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

2196it [04:22,  8.37it/s]


2196
Data has been written to ./mix_textvqa_data.json


In [13]:
mix_ocr_vqa_data = []
for index, data in tqdm(enumerate(ocr_vqa_data)):
    
    image_list = find_companions(data['image'], ocr_vqa_image, 1)
    in_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + data['image']
    out_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + image_list[0]

    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_pics_in_pics/ocrvqa_{index}.jpg'
    resize_and_paste_image(in_pics, out_pics, output_path)

    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = '<image>\n' + local_meta_prompt + ', ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = local_meta_prompt + ', ' +  convs['value'][0].lower() + convs['value'][1:]

    data['image'] = output_path
    mix_ocr_vqa_data.append(data)

print(len(mix_ocr_vqa_data))
output_json_path = "./mix_ocr_vqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_ocr_vqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

8000it [06:15, 21.30it/s]


8000
Data has been written to ./mix_ocr_vqa_data.json


In [14]:
mix_gqa_data = []
for index, data in tqdm(enumerate(gqa_data)):
    
    image_list = find_companions(data['image'], gqa_image, 1)
    in_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + data['image']
    out_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + image_list[0]

    output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_pics_in_pics/gqa_{index}.jpg'
    resize_and_paste_image(in_pics, out_pics, output_path)

    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
    data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
    for convs_index, convs in enumerate(data["conversations"]):
        if convs['from'] == 'human':
            if convs_index == 0:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = '<image>\n' + local_meta_prompt + ', ' + convs['value'][0].lower() + convs['value'][1:]
            else:
                local_meta_prompt = random.choice(meta_prompt)
                convs['value'] = local_meta_prompt + ', ' +  convs['value'][0].lower() + convs['value'][1:]

    data['image'] = output_path
    mix_gqa_data.append(data)

print(len(mix_gqa_data))
output_json_path = "./mix_gqa_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_gqa_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

7214it [07:37, 15.76it/s]


7214
Data has been written to ./mix_gqa_data.json


In [15]:
import re
def contains_bbox(s):
    bbox_pattern = r'\[\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?,\s*\d+(\.\d+)?\s*\]'
    match = re.search(bbox_pattern, s)
    if match:
        return True
    else:
        return False

In [16]:
mix_coco_data = []
for index, data in tqdm(enumerate(coco_data)):
    choose_flase = True
    for conv in data["conversations"]:
        conv_value = conv["value"]
        if contains_bbox(conv_value):
            choose_flase = False
    if choose_flase == True:
        image_list = find_companions(data['image'], coco_image, 1)
        in_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + data['image']
        out_pics = '/mnt/petrelfs/liuziyu/V3Det/LLaVA/scripts/v1_5/data/cl_data/' + image_list[0]
    
        output_path = f'/mnt/hwfile/mllm/liuziyu/RLHF_data/RLHF_pics_in_pics/coco_{index}.jpg'
        resize_and_paste_image(in_pics, out_pics, output_path)
    
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>\n', '')
        data["conversations"][0]['value'] = data["conversations"][0]['value'].replace('<image>', '')
        for convs_index, convs in enumerate(data["conversations"]):
            if convs['from'] == 'human':
                if convs_index == 0:
                    local_meta_prompt = random.choice(meta_prompt)
                    convs['value'] = '<image>\n' + local_meta_prompt + ', ' + convs['value'][0].lower() + convs['value'][1:]
                else:
                    local_meta_prompt = random.choice(meta_prompt)
                    convs['value'] = local_meta_prompt + ', ' +  convs['value'][0].lower() + convs['value'][1:]
    
        data['image'] = output_path
        mix_coco_data.append(data)

print(len(mix_coco_data))
output_json_path = "./mix_coco_data.json"
with open(output_json_path, 'w') as json_file:
    json.dump(mix_coco_data, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

36410it [39:46, 15.26it/s]  


31565
Data has been written to ./mix_coco_data.json


In [17]:
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_coco_data.json", 'r', encoding='utf-8') as file:
    mix_coco_data = json.load(file)
print(len(mix_coco_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_gqa_data.json", 'r', encoding='utf-8') as file:
    mix_gqa_data = json.load(file)
print(len(mix_gqa_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_ocr_vqa_data.json", 'r', encoding='utf-8') as file:
    mix_ocr_vqa_data = json.load(file)
print(len(mix_ocr_vqa_data))
with open("/mnt/petrelfs/liuziyu/RLHF/make_data/data_randomsample_randompic_45k/mix_textvqa_data.json", 'r', encoding='utf-8') as file:
    mix_textvqa_data = json.load(file)
print(len(mix_textvqa_data))
pics_in_pics_62k = mix_coco_data+mix_gqa_data+mix_ocr_vqa_data+mix_textvqa_data
print(len(pics_in_pics_62k))
output_json_path = "./pics_in_pics_49k.json"
with open(output_json_path, 'w') as json_file:
    json.dump(pics_in_pics_62k, json_file, indent=4)
print(f"Data has been written to {output_json_path}")

31565
7214
8000
2196
48975
Data has been written to ./pics_in_pics_49k.json
